<a href="https://colab.research.google.com/github/BwireDorcas/Kenyan-Constitution-Chatbot/blob/main/constitution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install dotenv
!pip install langchain_community
!pip install langchain_huggingface
!pip install langchain_pinecone
!pip install pinecone
!pip install groq
!pip install langchain_groq
!pip install langchain.chains
!pip install fastapi
!pip install pydantic
!pip install streamlit
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [40]:
#import libraries
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import RetrievalQA
from pinecone import Pinecone, ServerlessSpec
from langchain_groq import ChatGroq
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.document_loaders import PyPDFLoader

In [6]:
#Load env
load_dotenv()
app = FastAPI()

In [7]:
#Read env. files
os.environ["GROQ_API_KEY"] = "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"
os.environ["PINECONE_API_KEY"] = "pcsk_59aKPg_P7kCfqbLuWFmmVknWidDyccJtitQ68GGaLmU9VLmdvvcfFLpHX2cjsUto5GfrMW"
os.environ["PINECONE_ENV"] = "us-east-1"

GROQ_API_KEY = os.environ["GROQ_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENV = os.environ["PINECONE_ENV"]

In [8]:
# Confirm the values are set
print("GROQ_API_KEY")
print("PINECONE_API_KEY")
print("PINECONE_ENV")


GROQ_API_KEY
PINECONE_API_KEY
PINECONE_ENV


In [9]:
from google.colab import files
uploaded = files.upload()


Saving constitution_of_kenya.pdf to constitution_of_kenya.pdf


In [10]:
#Load the PDF file
!pip install PyPDF
loader = PyPDFLoader("constitution_of_kenya.pdf")
documents = loader.load()
print("PDF loaded successfully")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 7.2 MB/s eta 0:00:00
PDF loaded successfully


In [11]:
#Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

In [13]:
print(f"Total chunks: {len(documents)}")


Total chunks: 492


In [14]:
#embedding model
embedding = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
)

<ipython-input-14-3383c8ad0da4>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [15]:
#Create a vector store using Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
#create index
index_name = "llama-text-embed-v2"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index = pc.Index(index_name)
print("Index created successfully")

Index created successfully


In [16]:
#Create a vector store

vector_store = PineconeVectorStore.from_documents(
    documents=documents,
    index_name=index_name,
    embedding=embedding,
    namespace="your_namespace"
)
print("Vector store created successfully")

Vector store created successfully


In [17]:
#Create a retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
    )
print("Retriever created successfully")


Retriever created successfully


In [18]:
#test the retriever
query = "What are the functions of the President according to the Kenyan Constitution?"
docs = retriever.get_relevant_documents(query)

# Print results
for i, doc in enumerate(docs):
    print(f"\n--- Document {i+1} ---\n")
    print(doc.page_content)


<ipython-input-18-02c19c96fb38>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)



--- Document 1 ---

Constitution of Kenya  Kenya
(d) promote respect for the diversity of the people and communities of Kenya; and
(e) ensure the protection of human rights and fundamental freedoms and the rule of law.
(3) The President shall not hold any other State or public ofﬁce.
132. Functions of the President
(1) The President shall—
(a) address the opening of each newly elected Parliament;
(b) address a special sitting of Parliament once every year and may address Parliament at any
other time; and
(c) once every year—
(i) report, in an address to the nation, on all the measures taken and the progress
achieved in the realisation of the national values, referred to in Article 10;
(ii) publish in the Gazette the details of the measures and progress under sub-paragraph
(i); and
(iii) submit a report for debate to the National Assembly on the progress made in fulﬁlling
the international obligations of the Republic.

--- Document 2 ---

Constitution of Kenya  Kenya
(d) promote respec

In [20]:
#Initialize the LLM
llm = ChatGroq(
    temperature = 0,
    GROQ_API_KEY = GROQ_API_KEY,
    model= "llama2-70b-4096"

)

/usr/local/lib/python3.11/dist-packages/langchain_groq/chat_models.py:364: UserWarning: WARNING! GROQ_API_KEY is not default parameter.
                    GROQ_API_KEY was transferred to model_kwargs.
                    Please confirm that GROQ_API_KEY is what you intended.
  warnings.warn(


In [21]:
from langchain.prompts import PromptTemplate
#Prompt Template
template = """[INST]<<SYS>> Use the following context from the Kenyan Constitution to answer the question.
Be precise, cite articles if relevant.
If a question lies outside the scope of the Constitution say:
'I am a chatbot that helps you with the Kenyan Constitution. I cannot answer that.'
{context}

Question: {question}
Answer:[/INST]
"""
qst_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [22]:
retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": qst_prompt}
)

In [23]:
#define your request body
class QueryRequest (BaseModel):
  question:str

In [24]:
#API endpoint
@app.post("/ask")
async def ask_question(req: QueryRequest):
  response = retrieval_chain({"query": req.question})
  return {
      "question": req.question,
      "answer": response["result"],
      "sources": [doc.page_content[:300] for doc in response["source_documents"]]
      }

In [32]:
!pip install fastapi uvicorn pyngrok --quiet
from pyngrok import ngrok


In [33]:
!ls

constitution_of_kenya.pdf  constitution.py  drive  sample_data


In [41]:
with open("constitution.py", "w") as f:
    f.write('''
app = FastAPI()

class QueryRequest(BaseModel):
    question: str
@app.post("/ask")
async def ask_question(request: QueryRequest):
    response = retrieval_chain({"query": request.question})
    return {
        "question": request.question,
        "answer": response["result"],
        "sources": [doc.page_content[:300] for doc in response["source_documents"]]
    }
''')
